# Data Science Project: Planning Stage

**Name**: JunHyun Kim  
**Student Number**: 25323270  
**Group**: 10  

In [1]:
import pandas as pd
import altair as alt

# Title: Predicting and Optimizing Server Resource Allocation Based on Player Activity

## **Introduction**

### **Dataset Description**

In this project, we used two primary datasets to analyze player activity and optimize server resource allocation. These datasets are:

1. **`players.csv`** – Contains information about the players, including their experience level, subscription status, and hours played.
2. **`sessions.csv`** – Contains session details, such as the start and end times of each player's session.

These datasets were used to answer the question: **"How can we predict and optimize server resource allocation based on the number of concurrent players in each session?"**

### **1. `players.csv` Dataset**

The `players.csv` dataset provides detailed information about the players, which includes their **experience level**, **subscription status**, **gameplay hours**, and basic demographic information.

#### Main columns in `players.csv`:
- **`experience`**: The experience level of the player (e.g., "Pro", "Veteran", "Regular", "Amateur").
- **`subscribe`**: A boolean value indicating whether the player has a subscription (`True` or `False`).
- **`hashedEmail`**: A unique identifier for the player, stored as a hashed email address.
- **`played_hours`**: The total number of hours the player has spent playing the game.
- **`name`**: The player's name.
- **`gender`**: The player's gender.
- **`age`**: The player's age.
- **`individualId`**: An identifier for the player, possibly an internal ID for tracking purposes.
- **`organizationName`**: The name of the player's organization (if applicable).

#### Key Information from `players.csv`:
- **Number of Rows**: 196 players.
- **Number of Columns**: 9 attributes per player.
- **Data Types**: The dataset contains both categorical (e.g., `experience`, `gender`) and numerical (e.g., `played_hours`, `age`) data.

This dataset is important for understanding **player behavior** and how factors like **experience** and **age** might influence the number of connections (active players) at different times of the day.

### **2. `sessions.csv` Dataset**

The `sessions.csv` dataset contains information about the sessions in which players were actively playing the game. It records **session start and end times**, and is crucial for determining how long players are active and when their sessions occur.

#### Main columns in `sessions.csv`:
- **`hashedEmail`**: The player's unique identifier (same as in `players.csv`).
- **`start_time`**: The start time of the player's session, recorded as a string in the format `DD/MM/YYYY HH:MM`.
- **`end_time`**: The end time of the player's session, also recorded as a string in the same format as `start_time`.
- **`original_start_time`**: A timestamp in Unix format, representing the session's start time.
- **`original_end_time`**: A timestamp in Unix format, representing the session's end time.

#### Key Information from `sessions.csv`:
- **Number of Rows**: 1535 session entries.
- **Number of Columns**: 5 attributes per session.
- **Data Types**: The dataset includes both categorical data (e.g., `hashedEmail`) and datetime data (e.g., `start_time`, `end_time`).

This dataset is critical for determining **when players are active**, and analyzing **peak hours** or **days of the week** for player activity. The session duration (calculated from `start_time` and `end_time`) will also be important for identifying **intense activity periods** and **server load**.

### **Data Cleaning and Wrangling**

The datasets were cleaned and preprocessed as follows:

- **Datetime Conversion**: The `start_time` and `end_time` columns in the `sessions.csv` dataset were converted to proper `datetime` format to allow time-based analysis.
- **Session Duration**: We calculated the **session duration** by subtracting the `start_time` from the `end_time` to get the total duration of each session in minutes.
- **Handling Missing Data**: Rows with missing `end_time` values were dropped, as the session duration could not be calculated without this information.
- **Feature Extraction**: We extracted the **hour** and **day of the week** from the `start_time` column to enable analysis of player activity patterns by time.

We then aggregated the data by **hour**, **day of the week**, and both **hour and day of the week** to identify trends in player activity, which is essential for understanding when to allocate server resources more efficiently.

---


In [2]:
# Load the datasets
players_df = pd.read_csv('players.csv')
sessions_df = pd.read_csv('sessions.csv')

# Display the first few rows of each dataset
print("First few rows of players.csv:")
display(players_df.head())

print("\nFirst few rows of sessions.csv:")
display(sessions_df.head())

First few rows of players.csv:


,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN



First few rows of sessions.csv:


,hashedEmail,start_time,end_time,original_start_time,original_end_time
0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,30/06/2024 18:12,30/06/2024 18:24,1.719770e+12,1.719770e+12
1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,17/06/2024 23:33,17/06/2024 23:46,1.718670e+12,1.718670e+12
2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,25/07/2024 17:34,25/07/2024 17:57,1.721930e+12,1.721930e+12
3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,25/07/2024 03:22,25/07/2024 03:58,1.721880e+12,1.721880e+12
4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,25/05/2024 16:01,25/05/2024 16:12,1.716650e+12,1.716650e+12


In [3]:
# Get the shape of the DataFrame
num_players = players_df.shape[0]
num_player_vars = players_df.shape[1]
print(f"Number of players: {num_players}")
print(f"Number of variables in players.csv: {num_player_vars}")

# Get the shape of the DataFrame
num_sessions = sessions_df.shape[0]
num_session_vars = sessions_df.shape[1]
print(f"Number of sessions: {num_sessions}")
print(f"Number of variables in sessions.csv: {num_session_vars}")

Number of players: 196
Number of variables in players.csv: 9
Number of sessions: 1535
Number of variables in sessions.csv: 5


In [4]:
# Display data types
print("Data types in players.csv:")
print(players_df.dtypes)

print("\nData types in sessions.csv:")
print(sessions_df.dtypes)

# Display missing values
print("\nMissing values in players.csv:")
print(players_df.isnull().sum())

print("\nMissing values in sessions.csv:")
print(sessions_df.isnull().sum())

Data types in players.csv:
experience           object
subscribe              bool
hashedEmail          object
played_hours        float64
name                 object
gender               object
age                   int64
individualId        float64
organizationName    float64
dtype: object

Data types in sessions.csv:
hashedEmail             object
start_time              object
end_time                object
original_start_time    float64
original_end_time      float64
dtype: object

Missing values in players.csv:
experience            0
subscribe             0
hashedEmail           0
played_hours          0
name                  0
gender                0
age                   0
individualId        196
organizationName    196
dtype: int64

Missing values in sessions.csv:
hashedEmail            0
start_time             0
end_time               2
original_start_time    0
original_end_time      2
dtype: int64


In [5]:
# Convert 'start_time' and 'end_time' to datetime format
sessions_df['start_time'] = pd.to_datetime(sessions_df['start_time'], format='%d/%m/%Y %H:%M')
sessions_df['end_time'] = pd.to_datetime(sessions_df['end_time'], format='%d/%m/%Y %H:%M')

# Drop rows with missing end times
sessions_df = sessions_df.dropna(subset=['end_time'])
# Calculate session duration in minutes
sessions_df['session_duration'] = (sessions_df['end_time'] - sessions_df['start_time']).dt.total_seconds() / 60

# Filter out sessions with zero duration
sessions_df = sessions_df[sessions_df['session_duration'] > 0]

# Extract day of the week and hour from start time
sessions_df['day_of_week'] = sessions_df['start_time'].dt.day_name()
sessions_df['hour'] = sessions_df['start_time'].dt.hour

# Reorder columns for consistency
sessions_df = sessions_df[['hashedEmail', 'start_time', 'end_time', 'day_of_week', 'hour', 'session_duration']]

# Mapping Day to numeric vaule, Start as Monday: 0
sessions_df['day_of_week_numeric'] = sessions_df['day_of_week'].replace({
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3,
    "Friday": 4,
    "Saturday": 5,
    "Sunday": 6
})

# Preview the cleaned and tidied data
sessions_df.head()

,hashedEmail,start_time,end_time,day_of_week,hour,session_duration,day_of_week_numeric
0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,2024-06-30 18:12:00,2024-06-30 18:24:00,Sunday,18,12.0,6
1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,2024-06-17 23:33:00,2024-06-17 23:46:00,Monday,23,13.0,0
2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,2024-07-25 17:34:00,2024-07-25 17:57:00,Thursday,17,23.0,3
3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,2024-07-25 03:22:00,2024-07-25 03:58:00,Thursday,3,36.0,3
4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,2024-05-25 16:01:00,2024-05-25 16:12:00,Saturday,16,11.0,5


In [6]:
# Create a list to store the expanded data
expanded_data = []

# Iterate through each session row
for index, row in sessions_df.iterrows():
    start_hour = row['start_time'].hour
    end_hour = row['end_time'].hour
    day_of_week = row['start_time'].day_name()
    
    # Add an entry for each hour within the session duration
    for hour in range(start_hour, end_hour + 1):
        expanded_data.append({'day_of_week': day_of_week, 'hour': hour})

# Convert the expanded data into a DataFrame
expanded_sessions_df = pd.DataFrame(expanded_data)

# Group by day_of_week and hour to count the number of connections
hourly_connections = expanded_sessions_df.groupby(['day_of_week', 'hour']).size().reset_index(name='connections')

# Mapping Day to numeric vaule, Start as Monday: 0
hourly_connections['day_of_week_numeric'] = hourly_connections['day_of_week'].replace({
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3,
    "Friday": 4,
    "Saturday": 5,
    "Sunday": 6
})

# Display the results
hourly_connections.head()

,day_of_week,hour,connections,day_of_week_numeric
0,Friday,0,16,4
1,Friday,1,21,4
2,Friday,2,27,4
3,Friday,3,41,4
4,Friday,4,32,4


In [7]:
# Check for missing values in the dataset
print("Missing values in sessions_df:")
print(hourly_connections.isnull().sum())

# Display basic statistical summary
print("\nStatistical summary of sessions_df:")
print(hourly_connections.describe())

# Check data types of each column
print("\nData types of sessions_df:")
print(hourly_connections.dtypes)


Missing values in sessions_df:
day_of_week            0
hour                   0
connections            0
day_of_week_numeric    0
dtype: int64

Statistical summary of sessions_df:
             hour  connections  day_of_week_numeric
count  153.000000   153.000000           153.000000
mean    11.405229    16.424837             2.993464
std      7.254253    12.146958             2.008196
min      0.000000     1.000000             0.000000
25%      5.000000     6.000000             1.000000
50%     11.000000    14.000000             3.000000
75%     18.000000    24.000000             5.000000
max     23.000000    50.000000             6.000000

Data types of sessions_df:
day_of_week            object
hour                    int64
connections             int64
day_of_week_numeric     int64
dtype: object


In [8]:
# Aggregate connections by hour
hourly_connections = sessions_df.groupby('hour').size().reset_index(name='connections')

# Aggregate connections by day of the week
daily_connections = sessions_df.groupby('day_of_week_numeric').size().reset_index(name='connections')

# Aggregate connections by both day of the week and hour
day_hour_connections = sessions_df.groupby(['day_of_week_numeric', 'hour']).size().reset_index(name='connections')

# Display aggregated data
print("Hourly Connections:")
print(hourly_connections.head())

print("\nDaily Connections:")
print(daily_connections.head())

print("\nDay-Hour Connections:")
print(day_hour_connections.head())


Hourly Connections:
   hour  connections
0     0          128
1     1           79
2     2          152
3     3          131
4     4          150

Daily Connections:
   day_of_week_numeric  connections
0                    0          207
1                    1          203
2                    2          210
3                    3          223
4                    4          181

Day-Hour Connections:
   day_of_week_numeric  hour  connections
0                    0     0           22
1                    0     1           11
2                    0     2           14
3                    0     3           19
4                    0     4           26


In [9]:
hour_chart = alt.Chart(hourly_connections).mark_line().encode(
    x=alt.X('hour:O', title='Hour of the Day', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('connections:Q', title='Number of Connections'),
    color=alt.value('steelblue')
).properties(
    title='Number of Connections by Hour of the Day'
)

hour_chart

alt.Chart(...)

In [10]:
# Replace day_of_week_numeric with day names
daily_connections['day_of_week'] = daily_connections['day_of_week_numeric'].replace({
    0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 
    4: 'Friday', 5: 'Saturday', 6: 'Sunday'
})

day_chart = alt.Chart(daily_connections).mark_bar().encode(
    x=alt.X('day_of_week:O', title='Day of the Week', sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']),
    y=alt.Y('connections:Q', title='Number of Connections'),
    color=alt.Color('connections:Q', scale=alt.Scale(scheme='viridis'))
).properties(
    title='Number of Connections by Day of the Week'
)

day_chart


alt.Chart(...)

In [11]:
day_hour_chart = alt.Chart(day_hour_connections).mark_circle(size=100).encode(
    x=alt.X('hour:O', title='Hour of the Day', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('day_of_week_numeric:O', 
            title='Day of the Week', 
            sort=[0, 1, 2, 3, 4, 5, 6],
            axis=alt.Axis(
                labelExpr="datum.value == 0 ? 'Monday' : "
                          "datum.value == 1 ? 'Tuesday' : "
                          "datum.value == 2 ? 'Wednesday' : "
                          "datum.value == 3 ? 'Thursday' : "
                          "datum.value == 4 ? 'Friday' : "
                          "datum.value == 5 ? 'Saturday' : 'Sunday'"
            )),
    size=alt.Size('connections:Q', title='Connections'),
    color=alt.Color('connections:Q', scale=alt.Scale(scheme='blues')),
).properties(
    title='Number of Connections by Day and Hour',
    width=800,
    height=400
)

day_hour_chart


alt.Chart(...)

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
import pandas as pd

# Define the preprocessor
preprocessor = make_column_transformer(
    (StandardScaler(), ['connections']),
    verbose_feature_names_out=False,
)

# Set the range for possible cluster numbers
ks = range(1, 10)

# Compute WSSD for each value of K using list comprehension
wssds = [
    make_pipeline(
        preprocessor,
        KMeans(n_clusters=k, random_state=42)
    ).fit(day_hour_connections)[1].inertia_
    for k in ks
]

# Create a dataframe to store K and WSSD values
elbow_df = pd.DataFrame({
    "k": ks,
    "wssd": wssds,
})

# Plot the Elbow Curve
elbow_plot = alt.Chart(elbow_df).mark_line(point=True).encode(
    x=alt.X("k:N", title="Number of Clusters"),
    y=alt.Y("wssd:Q", title="Total Within-Cluster Sum of Squares (WSSD)"),
    tooltip=["k", "wssd"]
).properties(
    title="Elbow Method for Optimal K"
)
elbow_plot


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to '

alt.Chart(...)

In [13]:
# Set optimal number of clusters
optimal_k = 3

# Create and fit the pipeline
clustering_pipeline = make_pipeline(
    preprocessor,
    KMeans(n_clusters=optimal_k, random_state=42)
)
clustering_pipeline.fit(day_hour_connections)

# Add cluster labels to the original dataframe
day_hour_connections['cluster'] = clustering_pipeline[1].labels_

# Define a mapping for descriptive cluster labels
cluster_mapping = {0: 'Low', 2: 'Medium', 1: 'High'}
day_hour_connections['density_label'] = day_hour_connections['cluster'].map(cluster_mapping)

# Display sample of labeled data
day_hour_connections[['hour', 'day_of_week_numeric', 'connections', 'density_label']].head()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,hour,day_of_week_numeric,connections,density_label
0,0,0,22,High
1,1,0,11,Medium
2,2,0,14,Medium
3,3,0,19,High
4,4,0,26,High


In [14]:
# Updated visualization with descriptive labels
cluster_plot = alt.Chart(day_hour_connections).mark_circle(size=60).encode(
    x=alt.X("hour:O", title="Hour of the Day"),
    y=alt.Y("connections:Q", title="Number of Connections"),
    color=alt.Color("density_label:N", title="Session Density"),
    tooltip=["hour", "day_of_week_numeric", "connections", "density_label"]
).properties(
    title="K-Means Clustering of Connection Density (Descriptive Labels)",
    width=800,
    height=400
)

cluster_plot


alt.Chart(...)

In [15]:
# Map point sizes to the density labels for visualization
point_size_mapping = {'Low': 40, 'Medium': 80, 'High': 120}
day_hour_connections['size'] = day_hour_connections['density_label'].map(point_size_mapping)

# Updated visualization
day_hour_cluster_chart = alt.Chart(day_hour_connections).mark_circle().encode(
    x=alt.X('hour:O', title='Hour of the Day', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('day_of_week_numeric:O', 
            title='Day of the Week', 
            sort=[0, 1, 2, 3, 4, 5, 6],
            axis=alt.Axis(labelExpr="datum.value == 0 ? 'Monday' : datum.value == 1 ? 'Tuesday' : datum.value == 2 ? 'Wednesday' : datum.value == 3 ? 'Thursday' : datum.value == 4 ? 'Friday' : datum.value == 5 ? 'Saturday' : 'Sunday'")
           ),
    color=alt.Color('density_label:N', title='Cluster'),
    size=alt.Size('size:Q', title='Point Size', legend=None),  # Use mapped sizes
    tooltip=['day_of_week_numeric', 'hour', 'density_label', 'connections']
).properties(
    title='Clustering Results: Day and Hour',
    width=800,
    height=400
)

day_hour_cluster_chart


alt.Chart(...)

In [16]:
# Cluster distribution summary
cluster_distribution = day_hour_connections['density_label'].value_counts().reset_index()
cluster_distribution.columns = ['Cluster', 'Count']

print("\nCluster Distribution:")
print(cluster_distribution)


Cluster Distribution:
  Cluster  Count
0  Medium     58
1     Low     48
2    High     32
